In [1]:
import os
import numpy as np
import pandas as pd
import hashlib
import gzip
import re

In [2]:
os.chdir("E:/CNSI_project/demo_and_data_3.0/Raw_data/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/")

# Verification of document integrity

In [3]:
def generate_sha256_str(input_string):
    sha_signature = hashlib.sha256(input_string.encode()).hexdigest()
    return sha_signature

In [4]:
def generate_sha256_file(file_path):
    sha256_hash = hashlib.sha256()
    with open(file_path, 'rb') as f:
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)
    return sha256_hash.hexdigest()

In [5]:
sha256_dict = {}
with open('SHA256SUMS.txt', 'r') as f:
    for line in f:
        line = line.strip()
        if '.txt' not in line:
            a,b = line.split(' ')
            sha256_dict[b] = a
sha256_dict

{'note/discharge.csv.gz': 'c194a975571df5e1c2486c094a52fef7cb3e183b03556cff0c23cea55878578c',
 'note/discharge_detail.csv.gz': 'b3e1218db50ace5c866afe7f4cd9fd457206d4ea122749f1afca57ce9a1b7a1a',
 'note/radiology.csv.gz': '2c09bc76ee12d5f3b8f572deb7218fbaf3e59b32eccfdae14686e05ae099eebb',
 'note/radiology_detail.csv.gz': '9d3f6c44183c55ef8a0c511a2c72612568ad457fc53c035c8443c28a8cc53773'}

In [6]:
i = 0
for each in list(sha256_dict.keys()):
    if generate_sha256_file(each) != sha256_dict[each]:
        print(f"File {each} is incomplete!")
        i += 1
    else:
        continue
if i == 0:
    print("All files are complete!")

All files are complete!


In [7]:
filename = './note/discharge.csv.gz'
with gzip.open(filename, 'rt', encoding='utf-8') as file:
    discharge = pd.read_csv(file)
discharge

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text
0,10000032-DS-21,10000032,22595853,DS,21,2180-05-07 00:00:00,2180-05-09 15:26:00,\nName: ___ Unit No: _...
1,10000032-DS-22,10000032,22841357,DS,22,2180-06-27 00:00:00,2180-07-01 10:15:00,\nName: ___ Unit No: _...
2,10000032-DS-23,10000032,29079034,DS,23,2180-07-25 00:00:00,2180-07-25 21:42:00,\nName: ___ Unit No: _...
3,10000032-DS-24,10000032,25742920,DS,24,2180-08-07 00:00:00,2180-08-10 05:43:00,\nName: ___ Unit No: _...
4,10000084-DS-17,10000084,23052089,DS,17,2160-11-25 00:00:00,2160-11-25 15:09:00,\nName: ___ Unit No: __...
...,...,...,...,...,...,...,...,...
331789,19999828-DS-6,19999828,29734428,DS,6,2147-08-04 00:00:00,2147-08-12 15:36:00,\nName: ___ Unit No: ___...
331790,19999828-DS-7,19999828,25744818,DS,7,2149-01-18 00:00:00,2149-01-19 07:03:00,\nName: ___ Unit No: ___...
331791,19999840-DS-20,19999840,26071774,DS,20,2164-07-28 00:00:00,2164-07-29 14:52:00,\nName: ___ Unit No: ___\...
331792,19999840-DS-21,19999840,21033226,DS,21,2164-09-17 00:00:00,2164-09-18 01:36:00,\nName: ___ Unit No: ___\...


In [8]:
text_list = ['Name','Unit No','Admission Date','Discharge Date','Date of Birth','Sex','Service','Allergies','Attending',
             'Chief Complaint', 'Major Surgical or Invasive Procedure','History of Present Illness','Past Medical History',
             'Social History','Family History', 'Physical Exam','VS','General','HEENT','Neck','CV','Lungs','Abdomen','GU',
             'Ext', 'Neuro','Discharge','PHYSICAL EXAMINATION', 'VS','General','HEENT','Neck','CV','Lungs','Abdomen','GU',
             'Ext','Neuro','Pertinent Results','CXR','U/S','Brief Hospital Course','diuretics','Medications on Admission',
             'Discharge Medications','Discharge Disposition','Discharge Condition','Mental Status','Level of Consciousness',
             'Activity Status', 'Discharge Instructions','Followup Instructions']

In [9]:
len(text_list)

52

In [10]:
text_list = ['Name','Unit No','Admission Date','Discharge Date','Date of Birth','Sex','Service','Allergies','Attending',
             'Chief Complaint','Major Surgical or Invasive Procedure','History of Present Illness','Past Medical History',
             'Social History','Family History','Physical Exam','Discharge','Medications on Admission','Discharge Medications',
             'Discharge Disposition','Discharge Condition','Discharge Instructions','Followup Instructions']

In [11]:
text_list = ['Name','Unit No','Admission Date','Discharge Date','Date of Birth','Sex','Service','Allergies','Attending',
             'Complaint','Major Surgical or Invasive Procedure','History of Present Illness','Past Medical History',
             'Social History','Family History']

In [12]:
def extract_content(record, keyword, keys):
    next_keywords = '|'.join([re.escape(k) for k in keys if k != keyword])
    pattern = keyword + r':(.*?)(?=' + next_keywords + '|$)'
    match = re.search(pattern, record, re.S | re.I)
    if match:
        return match.group(1).strip()
    else:
        return None

def create_dict(e):
    keys = ['Name','Unit No','Admission Date','Discharge Date','Date of Birth','Sex','Service','Allergies','Attending','Chief Complaint','Major Surgical or Invasive Procedure','History of Present Illness','Past Medical History','Social History','Family History']
    return {key: extract_content(e, key, keys) for key in keys}

In [13]:
data = []
a = discharge.loc[:,'text'].tolist()
jj = 0
for e in a:
    e = e.split('\n')
    e = [each for each in e if each != '' and each != ' ']
    e = [each.strip() for each in e]
    e = ' '.join(e)
    e = re.sub(r'\s*:\s*', ':', e)
    e = re.split(r'(?i)(Physical Exam:)', e)[0]
    
    data.append(create_dict(e))
    jj += 1
    if jj%10000 == 0:
        print(f"The data for the {jj}th cases has been scanned.")

df = pd.DataFrame(data)
df

The data for the 10000th cases has been scanned.
The data for the 20000th cases has been scanned.
The data for the 30000th cases has been scanned.
The data for the 40000th cases has been scanned.
The data for the 50000th cases has been scanned.
The data for the 60000th cases has been scanned.
The data for the 70000th cases has been scanned.
The data for the 80000th cases has been scanned.
The data for the 90000th cases has been scanned.
The data for the 100000th cases has been scanned.
The data for the 110000th cases has been scanned.
The data for the 120000th cases has been scanned.
The data for the 130000th cases has been scanned.
The data for the 140000th cases has been scanned.
The data for the 150000th cases has been scanned.
The data for the 160000th cases has been scanned.
The data for the 170000th cases has been scanned.
The data for the 180000th cases has been scanned.
The data for the 190000th cases has been scanned.
The data for the 200000th cases has been scanned.
The data 

,Name,Unit No,Admission Date,Discharge Date,Date of Birth,Sex,Service,Allergies,Attending,Chief Complaint,Major Surgical or Invasive Procedure,History of Present Illness,Past Medical History,Social History,Family History
0,___,___,___,___,___,F,MEDICINE,No Known Allergies / Adverse Drug Reactions,___,Worsening ABD distension and pain,Paracentesis,"___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",1. HCV Cirrhosis 2. No history of abnormal Pap...,___,"She a total of five siblings, but she is not t..."
1,___,___,___,___,___,F,MEDICINE,Percocet,___.,abdominal fullness and discomfort,___ diagnostic paracentesis ___ therapeutic pa...,"___ with HIV on HAART, COPD, HCV cirrhosis com...",1. HCV Cirrhosis 2. No history of abnormal Pap...,___,"She a total of five siblings, but she is not ..."
2,___,___,___,___,___,F,MEDICINE,Percocet / Vicodin,___,altered mental status,none,"Mrs. ___ is a ___ female with HIV on HAART, CO...","- HCV Cirrhosis:genotype 3a - HIV:on HAART, __...",___,"She a total of five siblings, but she is not ..."
3,___,___,___,___,___,F,MEDICINE,Percocet / Vicodin,___.,Abdominal pain,___ Paracentesis,"___ w/ HIV on HAART, COPD on 3L home O2 (thoug...","- HCV Cirrhosis:genotype 3a - HIV:on HAART, __...",___,"She a total of five siblings, but she is not ..."
4,___,___,___,___,___,M,MEDICINE,No Known Allergies / Adverse Drug Reactions,___.,Visual hallucinations,N/A,"___ male with ___ disease, dyslipidemia, and a...",___ disease ___ Body Dementia dyslipidemia pro...,___,"His mother died at age ___ of ""old age."" His f..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331789,___,___,___,___,___,F,SURGERY,Lamictal / hydrochlorothiazide,___.,Enterocutaneous/enteroatmospheric fistula,"___:1. Split-thickness skin graft, left and r...","Ms. ___ is a ___ F w/hx of Afib on diltizaem, ...",PMH a fib on diltiazem Factor V Leiden deficie...,___,- both parents and multiple siblings have DVTs...
331790,___,___,___,___,___,F,MEDICINE,Lamictal / hydrochlorothiazide,___.,None,___ Debridement of intra-abdominal fluid colle...,None,None,None,None
331791,___,___,___,___,___,M,NEUROLOGY,Patient recorded as having No Known Allergies ...,___.,seizure,EEG,___ man with previous history of stroke and se...,-Left hemispheric stroke in ___ -Epilepsy sinc...,___,non-contributory
331792,___,___,___,___,___,M,MEDICINE,Patient recorded as having No Known Allergies ...,___.,seizures,intubation ___,Mr. ___ is a ___ year old RHM with prior L MCA...,-Left hemispheric stroke in ___ -Epilepsy sinc...,___,non-contributory


In [18]:
df_merged = pd.concat([discharge,df],axis=1)
df_merged

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text,Name,Unit No,...,Sex,Service,Allergies,Attending,Chief Complaint,Major Surgical or Invasive Procedure,History of Present Illness,Past Medical History,Social History,Family History
0,10000032-DS-21,10000032,22595853,DS,21,2180-05-07 00:00:00,2180-05-09 15:26:00,\nName: ___ Unit No: _...,___,___,...,F,MEDICINE,No Known Allergies / Adverse Drug Reactions,___,Worsening ABD distension and pain,Paracentesis,"___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",1. HCV Cirrhosis 2. No history of abnormal Pap...,___,"She a total of five siblings, but she is not t..."
1,10000032-DS-22,10000032,22841357,DS,22,2180-06-27 00:00:00,2180-07-01 10:15:00,\nName: ___ Unit No: _...,___,___,...,F,MEDICINE,Percocet,___.,abdominal fullness and discomfort,___ diagnostic paracentesis ___ therapeutic pa...,"___ with HIV on HAART, COPD, HCV cirrhosis com...",1. HCV Cirrhosis 2. No history of abnormal Pap...,___,"She a total of five siblings, but she is not ..."
2,10000032-DS-23,10000032,29079034,DS,23,2180-07-25 00:00:00,2180-07-25 21:42:00,\nName: ___ Unit No: _...,___,___,...,F,MEDICINE,Percocet / Vicodin,___,altered mental status,none,"Mrs. ___ is a ___ female with HIV on HAART, CO...","- HCV Cirrhosis:genotype 3a - HIV:on HAART, __...",___,"She a total of five siblings, but she is not ..."
3,10000032-DS-24,10000032,25742920,DS,24,2180-08-07 00:00:00,2180-08-10 05:43:00,\nName: ___ Unit No: _...,___,___,...,F,MEDICINE,Percocet / Vicodin,___.,Abdominal pain,___ Paracentesis,"___ w/ HIV on HAART, COPD on 3L home O2 (thoug...","- HCV Cirrhosis:genotype 3a - HIV:on HAART, __...",___,"She a total of five siblings, but she is not ..."
4,10000084-DS-17,10000084,23052089,DS,17,2160-11-25 00:00:00,2160-11-25 15:09:00,\nName: ___ Unit No: __...,___,___,...,M,MEDICINE,No Known Allergies / Adverse Drug Reactions,___.,Visual hallucinations,N/A,"___ male with ___ disease, dyslipidemia, and a...",___ disease ___ Body Dementia dyslipidemia pro...,___,"His mother died at age ___ of ""old age."" His f..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331789,19999828-DS-6,19999828,29734428,DS,6,2147-08-04 00:00:00,2147-08-12 15:36:00,\nName: ___ Unit No: ___...,___,___,...,F,SURGERY,Lamictal / hydrochlorothiazide,___.,Enterocutaneous/enteroatmospheric fistula,"___:1. Split-thickness skin graft, left and r...","Ms. ___ is a ___ F w/hx of Afib on diltizaem, ...",PMH a fib on diltiazem Factor V Leiden deficie...,___,- both parents and multiple siblings have DVTs...
331790,19999828-DS-7,19999828,25744818,DS,7,2149-01-18 00:00:00,2149-01-19 07:03:00,\nName: ___ Unit No: ___...,___,___,...,F,MEDICINE,Lamictal / hydrochlorothiazide,___.,None,___ Debridement of intra-abdominal fluid colle...,None,None,None,None
331791,19999840-DS-20,19999840,26071774,DS,20,2164-07-28 00:00:00,2164-07-29 14:52:00,\nName: ___ Unit No: ___\...,___,___,...,M,NEUROLOGY,Patient recorded as having No Known Allergies ...,___.,seizure,EEG,___ man with previous history of stroke and se...,-Left hemispheric stroke in ___ -Epilepsy sinc...,___,non-contributory
331792,19999840-DS-21,19999840,21033226,DS,21,2164-09-17 00:00:00,2164-09-18 01:36:00,\nName: ___ Unit No: ___\...,___,___,...,M,MEDICINE,Patient recorded as having No Known Allergies ...,___.,seizures,intubation ___,Mr. ___ is a ___ year old RHM with prior L MCA...,-Left hemispheric stroke in ___ -Epilepsy sinc...,___,non-contributory


In [19]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331794 entries, 0 to 331793
Data columns (total 23 columns):
 #   Column                                Non-Null Count   Dtype 
---  ------                                --------------   ----- 
 0   note_id                               331794 non-null  object
 1   subject_id                            331794 non-null  int64 
 2   hadm_id                               331794 non-null  int64 
 3   note_type                             331794 non-null  object
 4   note_seq                              331794 non-null  int64 
 5   charttime                             331794 non-null  object
 6   storetime                             331777 non-null  object
 7   text                                  331794 non-null  object
 8   Name                                  331794 non-null  object
 9   Unit No                               324364 non-null  object
 10  Admission Date                        331783 non-null  object
 11  Discharge Dat

In [20]:
useful_cols = ['hadm_id','Allergies','Chief Complaint','History of Present Illness','Past Medical History','Family History']

In [21]:
df_merged = df_merged[useful_cols]
df_merged

,hadm_id,Allergies,Chief Complaint,History of Present Illness,Past Medical History,Family History
0,22595853,No Known Allergies / Adverse Drug Reactions,Worsening ABD distension and pain,"___ HCV cirrhosis c/b ascites, hiv on ART, h/o...",1. HCV Cirrhosis 2. No history of abnormal Pap...,"She a total of five siblings, but she is not t..."
1,22841357,Percocet,abdominal fullness and discomfort,"___ with HIV on HAART, COPD, HCV cirrhosis com...",1. HCV Cirrhosis 2. No history of abnormal Pap...,"She a total of five siblings, but she is not ..."
2,29079034,Percocet / Vicodin,altered mental status,"Mrs. ___ is a ___ female with HIV on HAART, CO...","- HCV Cirrhosis:genotype 3a - HIV:on HAART, __...","She a total of five siblings, but she is not ..."
3,25742920,Percocet / Vicodin,Abdominal pain,"___ w/ HIV on HAART, COPD on 3L home O2 (thoug...","- HCV Cirrhosis:genotype 3a - HIV:on HAART, __...","She a total of five siblings, but she is not ..."
4,23052089,No Known Allergies / Adverse Drug Reactions,Visual hallucinations,"___ male with ___ disease, dyslipidemia, and a...",___ disease ___ Body Dementia dyslipidemia pro...,"His mother died at age ___ of ""old age."" His f..."
...,...,...,...,...,...,...
331789,29734428,Lamictal / hydrochlorothiazide,Enterocutaneous/enteroatmospheric fistula,"Ms. ___ is a ___ F w/hx of Afib on diltizaem, ...",PMH a fib on diltiazem Factor V Leiden deficie...,- both parents and multiple siblings have DVTs...
331790,25744818,Lamictal / hydrochlorothiazide,None,None,None,None
331791,26071774,Patient recorded as having No Known Allergies ...,seizure,___ man with previous history of stroke and se...,-Left hemispheric stroke in ___ -Epilepsy sinc...,non-contributory
331792,21033226,Patient recorded as having No Known Allergies ...,seizures,Mr. ___ is a ___ year old RHM with prior L MCA...,-Left hemispheric stroke in ___ -Epilepsy sinc...,non-contributory


# Merge the previous aim1_2 data

In [22]:
aim1_2 = pd.read_csv("E:/CNSI_project/demo_and_data_3.0/Processed_data/mimic/AddHadm/aim1_2_AddHadm_baidu.csv")
aim1_2

,subject_id,hadm_id,chartdate,Blood Gas~Blood~Alveolar-arterial Gradient~50801~mm Hg,Blood Gas~Blood~Base Excess~50802~mEq/L,"Blood Gas~Blood~Calculated Bicarbonate, Whole Blood~50803~mEq/L",Blood Gas~Blood~Calculated Total CO2~50804~mEq/L,Blood Gas~Blood~Carboxyhemoglobin~50805~%,"Blood Gas~Blood~Chloride, Whole Blood~50806~mEq/L",Blood Gas~Blood~Free Calcium~50808~mmol/L,...,icd_title,chiefcomplaint,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity
0,10001217,24597018.0,2157-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"OTHER MALAISE AND FATIGUE; FEVER, UNSPECIFIED",LEFT HAND PAINS,99.0,81.0,16.0,97.0,160.0,102.0,0.0,3.0
1,10002557,NaN,2147-03-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10002559,NaN,2179-06-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10003019,21616816.0,2174-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"FEVER, UNSPECIFIED; ALTERED MENTAL STATUS",FEVER/MS CHANGES,101.9,110.0,16.0,97.0,124.0,58.0,13.0,2.0
4,10005808,26543907.0,2119-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11854,19994772,NaN,2181-03-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11855,19995080,23057109.0,2147-07-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11856,19997843,NaN,2120-11-18,NaN,-2.0,NaN,22.0,NaN,NaN,1.16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11857,19999784,24935234.0,2119-07-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Weakness,Abnormal MRI,98.0,91.0,16.0,99.0,148.0,90.0,5.0,2.0


In [24]:
pd.set_option('display.max_columns',100)
aim1_2_merged = aim1_2.merge(df_merged, on=['hadm_id'], how='left')
aim1_2_merged

,subject_id,hadm_id,chartdate,Blood Gas~Blood~Alveolar-arterial Gradient~50801~mm Hg,Blood Gas~Blood~Base Excess~50802~mEq/L,"Blood Gas~Blood~Calculated Bicarbonate, Whole Blood~50803~mEq/L",Blood Gas~Blood~Calculated Total CO2~50804~mEq/L,Blood Gas~Blood~Carboxyhemoglobin~50805~%,"Blood Gas~Blood~Chloride, Whole Blood~50806~mEq/L",Blood Gas~Blood~Free Calcium~50808~mmol/L,Blood Gas~Blood~Glucose~50809~mg/dL,"Blood Gas~Blood~Hematocrit, Calculated~50810~%",Blood Gas~Blood~Hemoglobin~50811~g/dL,Blood Gas~Blood~Lactate~50813~mmol/L,Blood Gas~Blood~Methemoglobin~50814~%,Blood Gas~Blood~O2 Flow~50815~L/min,Blood Gas~Blood~Oxygen~50816~%,Blood Gas~Blood~Oxygen Saturation~50817~%,Blood Gas~Blood~pCO2~50818~mm Hg,Blood Gas~Blood~PEEP~50819~,Blood Gas~Blood~pH~50820~units,Blood Gas~Blood~pO2~50821~mm Hg,"Blood Gas~Blood~Potassium, Whole Blood~50822~mEq/L",Blood Gas~Blood~Required O2~50823~,"Blood Gas~Blood~Sodium, Whole Blood~50824~mEq/L",Blood Gas~Blood~Temperature~50825~,Blood Gas~Blood~Tidal Volume~50826~,Chemistry~Blood~% Hemoglobin A1c~50852~%,Chemistry~Blood~25-OH Vitamin D~50853~ng/mL,Chemistry~Blood~Acetaminophen~50856~ug/mL,Chemistry~Blood~Alanine Aminotransferase (ALT)~50861~IU/L,Chemistry~Blood~Albumin~50862~g/dL,Chemistry~Blood~Alkaline Phosphatase~50863~IU/L,Chemistry~Blood~Alpha-Fetoprotein~50864~ng/mL,Chemistry~Blood~Amikacin~50865~ug/mL,Chemistry~Blood~Ammonia~50866~umol/L,Chemistry~Blood~Amylase~50867~IU/L,Chemistry~Blood~Anion Gap~50868~mEq/L,Chemistry~Blood~Anti-DGP (IgA/IgG)~50869~units,"Chemistry~Blood~Anti-Gliadin Antibody, IgA~50870~units",Chemistry~Blood~Anti-Thyroglobulin Antibodies~50877~IU/mL,Chemistry~Blood~Asparate Aminotransferase (AST)~50878~IU/L,Chemistry~Blood~Beta-2 Microglobulin~50881~mg/L,Chemistry~Blood~Bicarbonate~50882~mEq/L,"Chemistry~Blood~Bilirubin, Direct~50883~mg/dL","Chemistry~Blood~Bilirubin, Indirect~50884~mg/dL","Chemistry~Blood~Bilirubin, Total~50885~mg/dL",Chemistry~Blood~C-Reactive Protein~50889~mg/L_,Chemistry~Blood~C3~50890~mg/dL,Chemistry~Blood~C4~50891~mg/dL,...,Hematology~Blood~Absolute Neutrophil Count~52075~K/uL_,Hematology~Blood~dRVVT - Confirmation~52107~,Hematology~Blood~dRVVT - Normalized Ratio~52108~,Hematology~Blood~dRVVT - Screen~52109~,Hematology~Blood~Immature Granulocytes~52135~%,Hematology~Blood~RDW-SD~52172~fL_,Hematology~Blood~SCT - Confirmation~52177~,Hematology~Blood~SCT - Normalized Ratio~52178~,Hematology~Blood~SCT - Screen~52179~,Hematology~Cerebrospinal Fluid~Atypical Lymphocytes~52221~%,Hematology~Cerebrospinal Fluid~Bands~52223~%,Hematology~Cerebrospinal Fluid~Basophils~52225~%,Hematology~Cerebrospinal Fluid~Blasts~52227~%,Hematology~Cerebrospinal Fluid~Eosinophils~52256~%,"Hematology~Cerebrospinal Fluid~Hematocrit, CSF~52257~%",Hematology~Cerebrospinal Fluid~Lining Cell~52263~%,Hematology~Cerebrospinal Fluid~Lymphs~52264~%,Hematology~Cerebrospinal Fluid~Macrophage~52266~%,Hematology~Cerebrospinal Fluid~Mesothelial cells~52268~%,Hematology~Cerebrospinal Fluid~Metamyelocytes~52270~%,Hematology~Cerebrospinal Fluid~Monocytes~52272~%,Hematology~Cerebrospinal Fluid~Myelocytes~52274~%,Hematology~Cerebrospinal Fluid~NRBC~52276~%,Hematology~Cerebrospinal Fluid~Other~52278~%,Hematology~Cerebrospinal Fluid~Plasma~52279~%,Hematology~Cerebrospinal Fluid~Polys~52281~%,Hematology~Cerebrospinal Fluid~Promyelocytes~52283~%,"Hematology~Cerebrospinal Fluid~RBC, CSF~52285~#/uL","Hematology~Cerebrospinal Fluid~Total Nucleated Cells, CSF~52286~#/uL",Hematology~Cerebrospinal Fluid~Young~52288~%,Hematology~Blood~Absolute Lymphocyte Count~52769~#/uL,gender,age,culture,long_title,icd_title,chiefcomplaint,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,Allergies,Chief Complaint,History of Present Illness,Past Medical History,Family History
0,10001217,24597018.0,2157-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,N

In [25]:
aim1_2_merged.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11859 entries, 0 to 11858
Columns: 312 entries, subject_id to Family History
dtypes: float64(299), int64(3), object(10)
memory usage: 28.3+ MB


# add patient outcomes

In [26]:
admission = pd.read_csv("E:/CNSI_project/demo_and_data_3.0/Raw_data/mimic-iv-2.2/hosp/admissions2.2.csv")
admission

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431226,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaN,EW EMER.,P75BG6,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0
431227,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,NaN,EW EMER.,P16C7J,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2147-07-17 17:18:00,2147-07-18 17:34:00,0
431228,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,EW EMER.,P58A9J,EMERGENCY ROOM,DIED,Other,ENGLISH,WIDOWED,WHITE,2164-09-10 11:09:00,2164-09-10 14:46:00,1
431229,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,NaN,EW EMER.,P506DE,EMERGENCY ROOM,HOME,Other,ENGLISH,WIDOWED,WHITE,2164-07-24 21:16:00,2164-07-25 01:20:00,0


In [27]:
aim1_2_merged1 = aim1_2_merged.merge(admission[['hadm_id','hospital_expire_flag']], on=['hadm_id'], how='left')
aim1_2_merged1

,subject_id,hadm_id,chartdate,Blood Gas~Blood~Alveolar-arterial Gradient~50801~mm Hg,Blood Gas~Blood~Base Excess~50802~mEq/L,"Blood Gas~Blood~Calculated Bicarbonate, Whole Blood~50803~mEq/L",Blood Gas~Blood~Calculated Total CO2~50804~mEq/L,Blood Gas~Blood~Carboxyhemoglobin~50805~%,"Blood Gas~Blood~Chloride, Whole Blood~50806~mEq/L",Blood Gas~Blood~Free Calcium~50808~mmol/L,Blood Gas~Blood~Glucose~50809~mg/dL,"Blood Gas~Blood~Hematocrit, Calculated~50810~%",Blood Gas~Blood~Hemoglobin~50811~g/dL,Blood Gas~Blood~Lactate~50813~mmol/L,Blood Gas~Blood~Methemoglobin~50814~%,Blood Gas~Blood~O2 Flow~50815~L/min,Blood Gas~Blood~Oxygen~50816~%,Blood Gas~Blood~Oxygen Saturation~50817~%,Blood Gas~Blood~pCO2~50818~mm Hg,Blood Gas~Blood~PEEP~50819~,Blood Gas~Blood~pH~50820~units,Blood Gas~Blood~pO2~50821~mm Hg,"Blood Gas~Blood~Potassium, Whole Blood~50822~mEq/L",Blood Gas~Blood~Required O2~50823~,"Blood Gas~Blood~Sodium, Whole Blood~50824~mEq/L",Blood Gas~Blood~Temperature~50825~,Blood Gas~Blood~Tidal Volume~50826~,Chemistry~Blood~% Hemoglobin A1c~50852~%,Chemistry~Blood~25-OH Vitamin D~50853~ng/mL,Chemistry~Blood~Acetaminophen~50856~ug/mL,Chemistry~Blood~Alanine Aminotransferase (ALT)~50861~IU/L,Chemistry~Blood~Albumin~50862~g/dL,Chemistry~Blood~Alkaline Phosphatase~50863~IU/L,Chemistry~Blood~Alpha-Fetoprotein~50864~ng/mL,Chemistry~Blood~Amikacin~50865~ug/mL,Chemistry~Blood~Ammonia~50866~umol/L,Chemistry~Blood~Amylase~50867~IU/L,Chemistry~Blood~Anion Gap~50868~mEq/L,Chemistry~Blood~Anti-DGP (IgA/IgG)~50869~units,"Chemistry~Blood~Anti-Gliadin Antibody, IgA~50870~units",Chemistry~Blood~Anti-Thyroglobulin Antibodies~50877~IU/mL,Chemistry~Blood~Asparate Aminotransferase (AST)~50878~IU/L,Chemistry~Blood~Beta-2 Microglobulin~50881~mg/L,Chemistry~Blood~Bicarbonate~50882~mEq/L,"Chemistry~Blood~Bilirubin, Direct~50883~mg/dL","Chemistry~Blood~Bilirubin, Indirect~50884~mg/dL","Chemistry~Blood~Bilirubin, Total~50885~mg/dL",Chemistry~Blood~C-Reactive Protein~50889~mg/L_,Chemistry~Blood~C3~50890~mg/dL,Chemistry~Blood~C4~50891~mg/dL,...,Hematology~Blood~dRVVT - Confirmation~52107~,Hematology~Blood~dRVVT - Normalized Ratio~52108~,Hematology~Blood~dRVVT - Screen~52109~,Hematology~Blood~Immature Granulocytes~52135~%,Hematology~Blood~RDW-SD~52172~fL_,Hematology~Blood~SCT - Confirmation~52177~,Hematology~Blood~SCT - Normalized Ratio~52178~,Hematology~Blood~SCT - Screen~52179~,Hematology~Cerebrospinal Fluid~Atypical Lymphocytes~52221~%,Hematology~Cerebrospinal Fluid~Bands~52223~%,Hematology~Cerebrospinal Fluid~Basophils~52225~%,Hematology~Cerebrospinal Fluid~Blasts~52227~%,Hematology~Cerebrospinal Fluid~Eosinophils~52256~%,"Hematology~Cerebrospinal Fluid~Hematocrit, CSF~52257~%",Hematology~Cerebrospinal Fluid~Lining Cell~52263~%,Hematology~Cerebrospinal Fluid~Lymphs~52264~%,Hematology~Cerebrospinal Fluid~Macrophage~52266~%,Hematology~Cerebrospinal Fluid~Mesothelial cells~52268~%,Hematology~Cerebrospinal Fluid~Metamyelocytes~52270~%,Hematology~Cerebrospinal Fluid~Monocytes~52272~%,Hematology~Cerebrospinal Fluid~Myelocytes~52274~%,Hematology~Cerebrospinal Fluid~NRBC~52276~%,Hematology~Cerebrospinal Fluid~Other~52278~%,Hematology~Cerebrospinal Fluid~Plasma~52279~%,Hematology~Cerebrospinal Fluid~Polys~52281~%,Hematology~Cerebrospinal Fluid~Promyelocytes~52283~%,"Hematology~Cerebrospinal Fluid~RBC, CSF~52285~#/uL","Hematology~Cerebrospinal Fluid~Total Nucleated Cells, CSF~52286~#/uL",Hematology~Cerebrospinal Fluid~Young~52288~%,Hematology~Blood~Absolute Lymphocyte Count~52769~#/uL,gender,age,culture,long_title,icd_title,chiefcomplaint,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,Allergies,Chief Complaint,History of Present Illness,Past Medical History,Family History,hospital_expire_flag
0,10001217,24597018.0,2157-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [28]:
aim1_2_merged1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11859 entries, 0 to 11858
Columns: 313 entries, subject_id to hospital_expire_flag
dtypes: float64(300), int64(3), object(10)
memory usage: 28.4+ MB


In [29]:
aim1_2_merged1.count()

subject_id                                                11859
hadm_id                                                    7093
chartdate                                                 11859
Blood Gas~Blood~Alveolar-arterial Gradient~50801~mm Hg      121
Blood Gas~Blood~Base Excess~50802~mEq/L                    1524
                                                          ...  
Chief Complaint                                            6752
History of Present Illness                                 6902
Past Medical History                                       6897
Family History                                             6864
hospital_expire_flag                                       7093
Length: 313, dtype: int64

In [30]:
aim1_2_merged1['culture'].value_counts()

0    11582
1      277
Name: culture, dtype: int64

In [31]:
aim1_2_merged1['hospital_expire_flag'].value_counts()

0.0    6679
1.0     414
Name: hospital_expire_flag, dtype: int64

In [32]:
aim1_2_merged1.to_csv('E:/CNSI_project/demo_and_data_3.0/Processed_data/mimic/AddHadm/aim1_2_merged1_baidu.csv',index=False)